# A snapthrough problem

A cylindric tank with concave lid is loaded with internal pressure. The tank is made from soft, incompressible material that is described by a Neo-Hooke hyperelastic model. Under internal pressure, the concave lid snaps into a concave shape. At snapping, the equilibrium path is instable and results in a drop of internal pressure.

A radially symmetric model is used, such that a reliable axisymmetric solution can be computed at moderate computational complexity.

In [ ]:
from ngsolve import *
import numpy as np
import netgen.occ as ngocc
import netgen.geom2d as ng2d
from netgen.meshing import IdentificationType, MeshingParameters

from newtonmethod import NewtonWithLinesearch

from ngsolve.webgui import Draw
from netgen.webgui import Draw as ngDraw

import matplotlib.pyplot as plt

SetNumThreads(4)

The tank is modeled in an axisymmetric way. For visualization, see also the 3D model. The tank will be clamped at the hole in the bottom.

In [ ]:
height = 50
thickness = 0.5
radius = 30
radius_in = 3
radius_top = 100
m_top = height + np.sqrt(radius_top**2-radius**2)

E = 1
nu = 0.5

mu = E/2/(1+nu)

pressure = 0.0001


In [ ]:
def GenerateOCC2d():
    H = height
    R = radius
    t = thickness
    Rin = radius_in
    Rtop = radius_top

    # wp = ngocc.WorkPlane().LineTo(0,-t/2).LineTo(R+t/2,-t/2).LineTo(R+t/2,H+2*t).LineTo(0,H+2*t).Close()
    wp = ngocc.WorkPlane().MoveTo(0,-t/2).Rectangle(R+t/2,H+t+t)
    rect = wp.Face()
    wp = ngocc.WorkPlane().Circle(0,m_top,Rtop-t/2)
    circ = wp.Face()
    wp = ngocc.WorkPlane().LineTo(0,-t).LineTo(Rin,-t).LineTo(Rin,0).LineTo(Rin,t).LineTo(0,t).Close()
    inlet = wp.Face()

    exterior = rect-circ-inlet
    wp = ngocc.WorkPlane().MoveTo(0,t/2).Rectangle(R-t/2,H)
    rect1 = wp.Face()
    circ1 = ngocc.WorkPlane().Circle(0,m_top,Rtop+t/2).Face()
    interior = rect1-circ1

    for (i,e) in enumerate(interior.edges):
        e.name = f"interior{i}"
    for e in exterior.edges:
        e.name = "exterior"
        
    face = exterior-interior
    for e in face.edges:
        if e.center[0] < 1e-10 and e.center[1] > H/2:
            e.name = "axis"
        if (e.center[0]-Rin) < 1e-10 and e.center[1] < H/2:
            e.name = "fix"

    for p in face.vertices:
        if p.p[0] == Rin and p.p[1] == 0:
            p.name = "point0"

    ngDraw(face)

    tank3d = face.Revolve(ngocc.Axis((0,0,0),ngocc.Y),360)
    ngDraw(tank3d)

    
    geo = ngocc.OCCGeometry(face, dim=2)
    
    mp = MeshingParameters(maxh=t/3)

    ngmesh = geo.GenerateMesh(mp)


    mesh = Mesh(ngmesh)
    return geo, mesh

geo, mesh = GenerateOCC2d()


Internal pressure acting in direction normal to the surfaces is applied to the internal surfaces of the tank. In such a setting, a large deformation elasticity model including follower loads is necessary. We use common continuum-mechanics notation, with $\vec u$ the displacement field, $\vec X$ the material coordinates, and $\mathbf F = \mathbf I + \partial \vec u/\partial \vec X$ the deformation gradient. Let $\mathbf C = \mathbf F^T \cdot \mathbf F$ denote the right Cauchy-Green tensor. We assume an incompressible Neo-Hooke material law, which is modeled by its potential

$$
\psi = \frac{\mu}{2} (\operatorname{tr}\hat {\mathbf C} - 3) + p \log J,
$$

with $\hat {\mathbf C} = J^{-2/3} \mathbf C$ and $p$ an independent hydrostatic pressure.

For the discretization, we use elements of Taylor-Hood type, vector-valued $H^1$ of order $k$ for the displacement and scalar $H^1$ of order $k-1$ for the pressure, and set $k=3$.

The internal pressure is modeled in direction of the surface normal in spatial configuration. Let $\vec N$ be the surface normal in reference configuration, then 

$$\vec n = \frac{\operatorname{Cof}\mathbf F\cdot  \vec N}{|\operatorname{Cof}\mathbf F \cdot \vec N|} = \frac{1}{J_A} \operatorname{Cof}\mathbf F\cdot  \vec N$$

with $J_A$ the jacobian for the area element. We deduce, for a constant surface pressure $\tilde p$ in spatial configuration, i.e. on the deformed boundary part $\gamma$, we can transform

$$\int_\gamma \tilde p \vec n \cdot \delta \vec u\, da = \int_\Gamma \tilde p \operatorname{Cof}\mathbf F\cdot  \vec N \, dA.$$

The principle of virtual work, for the hyperelastic material law and internal pressure as above, reads

$$\int_\Omega \delta \psi\, dV = \int_\Gamma \tilde p \operatorname{Cof}\mathbf F\cdot  \vec N \, dA.$$

We anticipate instable behavior at snap-through point. Therefore, we rather prescribe the work dual, which is the mean normal displacement of the internal surfaces, with $\tilde p$ the Lagrange multiplier. In the finite element setup, this requires a single, independent unknown associated to these surfaces.

In [ ]:
order = 3

fespace_u = VectorH1(mesh, order=order, dirichletx="axis", dirichlet="fix")
fespace_p = H1(mesh, order=order-1)
fespace_num = FESpace("number", mesh, definedon=mesh.Boundaries("interior."))

fespace = fespace_u * fespace_p * fespace_num

In [ ]:
q = GridFunction(fespace)
u, p, pres = q.components

loadpar = Parameter(0)

In the axisymmetric setting, the formal coordinates are $x=r$ and $y=z$, which we transform to the three-dimensional material coordinates $X, Y, Z$ using the functionality below.'

In [ ]:
facJ = 2*np.pi*x

def Grad3D(U):
    if U.dim == 2:
        DURDR = Grad(U)[0,0]
        DURDZ = Grad(U)[0,1]
        DUZDR = Grad(U)[1,0]
        DUZDZ = Grad(U)[1,1]
        
        G = IfPos(x-1e-6, CoefficientFunction((DURDR, 0, DURDZ, 0, 1/(x)*U[0], 0, DUZDR, 0, DUZDZ), dims=(3,3)),
        CoefficientFunction((DURDR, 0, DURDZ, 0, DURDR, 0, DUZDR, 0, DUZDZ), dims=(3,3)) )

    else:
        DURDR = Grad(U)[0]
        DURDZ = Grad(U)[1]
    
        G = CoefficientFunction((DURDR, 0, DURDZ))

    return G

def Vec23D(U):
    return CoefficientFunction((U[0], 0, U[1]))


In [ ]:
u_, p_, pressure_ = fespace.TrialFunction()
deltau_, deltap_, deltapressure_ = fespace.TestFunction()

gradu_ = Grad3D(u_)
F_ = (Id(3) + gradu_)# *predef
C_ = F_.trans*F_
J_ = Det(F_)
hatC_ = J_**(-2/3)*C_
E_ = 0.5*(C_-Id(3))

Normal = Vec23D(specialcf.normal(2))
defgrad_surf_ = Id(3) + Grad3D(u_.Trace())
normal = Normalize(Cof(defgrad_surf_)*Normal)


a = BilinearForm(fespace)

a += SymbolicEnergy( ((mu/2*(Trace(hatC_)-3) + p_*log(J_))*facJ).Compile())
## some workaround as the normal direction is not always outward
a += SymbolicEnergy( (pressure_*(InnerProduct(Vec23D(u_), normal)+loadpar)*facJ).Compile(), definedon=mesh.Boundaries("interior1"))
a += SymbolicEnergy( (-pressure_*(InnerProduct(Vec23D(u_), normal)-loadpar)*facJ).Compile(), definedon=mesh.Boundaries("interior3|interior2"))

In [ ]:
scenep = Draw(p, mesh, deformation=u)

Increasing the prescribed mean normal displacement mimicks the process of inflation. Up to the snap-through point, the observed internal pressure $\tilde p$, which is the Lagrangian multiplier, increases. At snap-through, a large change in the deformation state occurs, the internal pressure drops instantly. A concave shape of the lid evolves spontaneously. But even before the actual snap-through, the pressure drops in a more controlled manner, due to the evolution of a higher-order convex shape in the lid.

In [ ]:
q_history = GridFunction(fespace, multidim=0)

In [ ]:
loadsteps1 = np.linspace(0,1,41,endpoint=True)
# loadsteps = np.concatenate((np.linspace(0,0.5,21,endpoint=True), np.linspace(0.5,0,21,endpoint=True)))
deflist = []
preslist = []
for l in loadsteps1[:]:
    loadpar.Set(l)
    print(f"loadfactor l = {l}")
    err, nit = NewtonWithLinesearch(a, q.vec, maxnewton=100)

    up = 1/Integrate(1, mesh, definedon=mesh.Boundaries("axis"))*Integrate(u[1], mesh, definedon=mesh.Boundaries("axis"))
    print(up, pres.vec[0])
    deflist.append(up)
    preslist.append(pres.vec[0])
    scenep.Redraw()

    q_history.AddMultiDimComponent(q.vec)
    if err: break

I thought that should work?? 

In [ ]:
Draw (q_history.components[0], mesh, animate=True, autoscale=True, deformation=True)

We visualize mean normal displacement over pressure, as well as the position of the tank's top point over internal pressure. The snap-through point is clearly visible.

In [ ]:
plt.plot(preslist, loadsteps1, "x", label="inflating")
plt.xlabel("internal pressure")
plt.ylabel("mean normal displacement")
plt.legend()

In [ ]:
plt.plot(preslist, deflist, "x", label="inflating")
plt.xlabel("internal pressure")
plt.ylabel("vertical displacement top")
plt.legend()

When decreasing the prescribed mean normal displacement, again snap-through is observed - however at a lower value than in the original inflation process. Starting from the inflated state at maximum internal pressure, the pressure decreases. When reaching the critical value from the inflation process, however, the pressure increases in a stable manner, until snapping back at a lower load factor.

In [ ]:
q_history2 = GridFunction(fespace, multidim=0)

loadsteps2 = np.linspace(1,0,41,endpoint=True)
deflist2 = []
preslist2 = []
for l in loadsteps2[:]:
    loadpar.Set(l)
    print(f"loadfactor l = {l}")
    err, nit = NewtonWithLinesearch(a, q.vec, maxnewton=100)

    up = 1/Integrate(1, mesh, definedon=mesh.Boundaries("axis"))*Integrate(u[1], mesh, definedon=mesh.Boundaries("axis"))
    print(up, pres.vec[0])
    deflist2.append(up)
    preslist2.append(pres.vec[0])
    scenep.Redraw()
    q_history2.AddMultiDimComponent(q.vec)

    if err: break

In [ ]:
Draw (q_history2.components[0], mesh, animate=True, autoscale=True, deformation=True)

In [ ]:
plt.plot(preslist, deflist, "x", label="inflating")
plt.plot(preslist2, deflist2, "+", label="deflating")
plt.xlabel("internal pressure")
plt.ylabel("vertical displacement top")
plt.legend()

In [ ]:
plt.plot(preslist, loadsteps1, "x", label="inflating")
plt.plot(preslist2, loadsteps2, "+", label="deflating")
plt.xlabel("internal pressure")
plt.ylabel("mean normal displacement")
plt.legend()